In [1]:
# Configuration
from dotenv import dotenv_values

config = dotenv_values(".env")

json_filename = f"{config['LEARN_COURSE_ID']}.json"

# https://www.drupal.org/docs/core-modules-and-themes/core-modules/jsonapi-module/api-overview
api_pages = f"{config['DRUPAL_API_URL']}/node/page"
api_courses = f"{config['DRUPAL_API_URL']}/group/course"
api_type_course = "group--course"
api_course_weeks = f"{config['DRUPAL_API_URL']}/node/course_week"
api_type_course_week = "node--course_week"
api_course_week_relationship = (
    f"{config['DRUPAL_API_URL']}/group_relationship/course-group_node-course_week"
)
api_type_course_week_relationship = "group_relationship--course-group_node-course_week"

In [2]:
# Authentication
from requests import session

with session() as session:
    session.auth = (config["DRUPAL_USERNAME"], config["DRUPAL_PASSWORD"])
    session.headers.update(
        {
            "Accept": "application/vnd.api+json",
            "Content-Type": "application/vnd.api+json",
        }
    )

In [3]:
# Load course json
from json import load

with open(file=json_filename, mode="r", encoding="utf-8") as f:
    course_json = load(f)

In [4]:
# Helper method for processing Learn JSON
def visit_children_recursively(results, item):
    # if isinstance(item, str):
    #     results.append(item)
    #     return
    if not item["availability"]["available"] == "Yes":
        return

    if "body" not in item:
        results.append(f"<h2>{item['title']}</h2>")
    else:
        if "title" in item:
            results.append(f"<h3>{item['title']}</h3>")
        results.append(item["body"])

    if "hasChildren" in item and item["hasChildren"] == True:
        for child in item["children"]:
            visit_children_recursively(results, child)

In [5]:
# Make course description out of Welcome and Course Information items
desc_items = []

for root_item in course_json["content"]:
    if root_item["title"] == "Welcome" or root_item["title"] == "Course Information":
        visit_children_recursively(desc_items, root_item)

course_description = "<br>".join(desc_items)

In [6]:
# Create new course page
from json import dumps

res = session.post(
    api_courses,
    data=dumps(
        {
            "data": {
                "type": api_type_course,
                "attributes": {
                    "label": course_json["details"]["name"],
                    "field_course_summary": {
                        "value": f"<p>{course_json['details']['courseId']}</p>",
                        "format": "basic_html",
                    },
                    "field_description": {
                        "value": course_description,
                        "format": "full_html",
                        "summary": "",
                    },
                },
            }
        }
    ),
)
course_uuid = res.json()["data"]["id"]

In [7]:
# Make course pages out of Course Materials items
materials_root_item = None
for root_item in course_json["content"]:
    if root_item["title"] == "Course Materials":
        materials_root_item = root_item
        break

all_materials_items = []
for materials_item in materials_root_item["children"]:
    subset_materials_items = []
    visit_children_recursively(subset_materials_items, materials_item)
    if len(subset_materials_items):
        all_materials_items.append(
            {
                "title": materials_item["title"],
                "content": "<br>".join(subset_materials_items),
            }
        )

In [8]:
# Create course week pages and attach them to course
for materials_item in all_materials_items:
    # First create the course week page
    res = session.post(
        api_course_weeks,
        data=dumps(
            {
                "data": {
                    "type": api_type_course_week,
                    "attributes": {
                        "title": materials_item["title"],
                        "body": {
                            "value": materials_item["content"],
                            "format": "full_html",
                            "summary": "",
                        },
                    },
                    "relationships": {
                        "field_back_to_course": {
                            "data": {
                                "type": api_type_course,
                                "id": course_uuid,
                            },
                        },
                    },
                }
            }
        ),
    )
    print(f"Added {materials_item['title']}")
    course_week_uuid = res.json()["data"]["id"]

    # Then create a relationship linking the new course week page to the course group
    res = session.post(
        api_course_week_relationship,
        data=dumps(
            {
                "data": {
                    "type": api_type_course_week_relationship,
                    "attributes": {
                        "label": all_materials_items[1]["title"],
                    },
                    "relationships": {
                        "entity_id": {
                            "data": {
                                "type": api_type_course_week,
                                "id": course_week_uuid,
                            }
                        },
                        "gid": {
                            "data": {
                                "type": api_type_course,
                                "id": course_uuid,
                            }
                        },
                    },
                }
            }
        ),
    )
    print(f"Connected new course week to {course_json['details']['name']}")
    # With Drupal 10.0.9 + Group 3.1.0 module, the attaching step doesn't work
    # It fails with HTTP 500 error: Drupal\\group\\Plugin\\Group\\RelationHandler\\EmptyAccessControl::relationshipCreateAccess(): Argument #1 ($group) must be of type Drupal\\group\\Entity\\GroupInterface, null given, called in drupal\\modules\\contrib\\group\\src\\Entity\\Access\\GroupRelationshipAccessControlHandler.php on line 61
    # This is a known bug: https://www.drupal.org/project/group/issues/2872645
    # Can be resolved by applying patch until official fix is released
    # Instructions here: https://www.drupal.org/docs/develop/git/using-git-to-contribute-to-drupal/working-with-patches/applying-a-patch-in-a-feature-branch
    # Add to composer.json > extra:
    # "patches": {
    #     "drupal/group": {
    #         "#2872645 - Creating Group content via JSON:API": "https://www.drupal.org/files/issues/2023-02-15/group-add-content-2872645-50.patch"
    #     }
    # }

Added Lecture Recordings
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 10: NLP and Neural Nets
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 9: Lexical Semantics
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 8: Semantics
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 7: Parsing
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 6: POS tagging and syntactic parsing
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 5: Strikes, Morphology on Friday
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 4: ML Methods and Strikes
Connected new course week to Foundations of Natural Language Processing (2022-2023)[SEM2]
Added Week 3: